 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("emails.csv")

In [3]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [85]:
def extract_message(string):
    start = string.find('FileName:')
    sub = string[start: ].find('\n')
    return string[start: ][sub:]

In [86]:
df['content'] = df.message.apply(extract_message)

In [92]:
df.content.head(5)

0                        \n\nHere is our forecast\n\n 
1    \n\nTraveling to have a business meeting takes...
2                   \n\ntest successful.  way to go!!!
3    \n\nRandy,\n\n Can you send me a schedule of t...
4              \n\nLet's shoot for Tuesday at 11:45.  
Name: content, dtype: object

In [122]:
import re
def getWords(text):
    return " ".join(re.compile('\w*[A-Za-z]\w*').findall(text))

In [108]:
from nltk.tokenize import TreebankWordTokenizer
def token(text):
    tokenizer = TreebankWordTokenizer()
    return tokenizer.tokenize(text)

In [101]:
df_test = df[1:50]

In [123]:
df_test['words'] = df_test.content.apply(getWords)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [109]:
df_test['words'] = df_test.content.apply(token)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [124]:
df_test.words.head()

1    Traveling to have a business meeting takes the...
2                            test successful way to go
3    Randy Can you send me a schedule of the salary...
4                           Let s shoot for Tuesday at
5    Greg How about either next Tuesday or Thursday...
Name: words, dtype: object

In [125]:
df['words'] = df.content.apply(getWords)

In [126]:
df.words.head()

0                                 Here is our forecast
1    Traveling to have a business meeting takes the...
2                            test successful way to go
3    Randy Can you send me a schedule of the salary...
4                           Let s shoot for Tuesday at
Name: words, dtype: object

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [128]:
n_features = 1000

In [129]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.words)

In [131]:
from sklearn.decomposition import LatentDirichletAllocation

In [134]:
n_topics = 30
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50.,
                                random_state=0)

In [ ]:
lda.fit(tf)

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
n_top_words = 20

In [ ]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)